In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re

from collections import namedtuple, Counter
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

## Посмотреть данные

### Посмотреть на игроков

In [ ]:
with open("players.pkl", 'rb') as f:
    players = pickle.load(f)

In [ ]:
# количество игроков
print(len(players.keys()))

In [ ]:
with open("players.pkl", 'rb') as f:
    players = pickle.load(f)

players[15]

inner_keys = set()

for key in players:
    for inner_key in players[key].keys():
        if inner_key not in inner_keys:
            inner_keys.add(inner_key)

In [ ]:
players_sample = np.random.choice(np.array(list(players.keys())), size=1000)

### Отобрать нужные записи и results

In [ ]:
%%time
with open("results.pkl", 'rb') as f:
    results = pickle.load(f)

In [ ]:
%%time
results_needed = {}

for key, lst in results.items():
    for lst_el in lst:
        if 'mask' in lst_el and lst_el['mask'] is not None \
                and 'teamMembers' in lst_el and lst_el['teamMembers']:
            results_needed[key] = lst
            break

In [ ]:
len(results.keys())

In [ ]:
len(results_needed.keys())

In [ ]:
with open("results_needed.pkl", 'wb') as f:
    pickle.dump(results_needed, f)

In [ ]:
%%time
del results
del results_needed

In [ ]:
%%time
with open("results_needed.pkl", 'rb') as f:
    results = pickle.load(f)

### Посчитать число записей с первичным ключём <br> турнир - команда - игрок - вопрос

In [ ]:
%%time
with open("tournaments.pkl", 'rb') as f:
    tournaments = pickle.load(f)

In [ ]:
len(tournaments.keys())

In [ ]:
3656 * 

In [ ]:
%%time
with open("results_needed.pkl", 'rb') as f:
    results = pickle.load(f)

In [ ]:
%%time
cnt_records = []
for tournament_id, lst in results.items():
    if lst is not None:
        for team_id, lst_el in enumerate(lst):
            if lst_el['teamMembers'] is not None and lst_el['mask'] is not None:
                to_add = (tournament_id, team_id, len(lst_el['teamMembers']), len(lst_el['mask']))
                cnt_records.append(to_add)

In [ ]:
del results

In [ ]:
sum([i[2] * i[3] for i in cnt_records])

In [ ]:
# только по командам x турнир x число вопросов
sum([i[3] for i in cnt_records])

In [ ]:
sum([i[2] for i in cnt_records])

### Разделить на train/test

In [ ]:
# определить стартовые даты турниров
# tournaments = set([i.tournament_id for i in plays])

with open("tournaments.pkl", 'rb') as f:
    tournaments_data = pickle.load(f)

tournaments_data = {key: value for key, value in tournaments_data.items() } # if key in tournaments}

tournaments_start_dates = {key: value['dateStart'] for key, value in tournaments_data.items()}

del tournaments_data

In [ ]:
%%time
with open("results_needed.pkl", 'rb') as f:
    results = pickle.load(f)

In [ ]:
# players_sample_set = set(players_sample)

In [ ]:
%%time
# собрать данные за 2019 год для train и за 2020 за тест по всем игрока
Records = namedtuple("Records", ("tournament_id", "t_start_date", "team_ind", "team_id", "players", "mask"))

plays_train = []
plays_test = []
for tournament_id, lst in results.items():
    if lst is not None:
        for team_ind, lst_el in enumerate(lst):
            if lst_el['teamMembers'] is not None and lst_el['mask'] is not None:
                # to_add = (tournament_id, team_id, len(lst_el['teamMembers']), len(lst_el['mask']))
                
                team_members = lst_el['teamMembers']
                players = [i['player']['id'] for i in team_members]
                team_id = lst_el['team']['id']
                mask = lst_el['mask']
                start_date = tournaments_start_dates[tournament_id]
                
                if pd.to_datetime(start_date).year == 2019:
                            # and any([i in players_sample_set for i in players]):
                    plays_train.append(Records(tournament_id=tournament_id, 
                                         t_start_date=start_date,
                                         team_ind=team_ind, 
                                         team_id=team_id, 
                                         players=players, 
                                         mask=mask
                                        )
                                )
                elif pd.to_datetime(start_date).year == 2020:
                    plays_test.append(Records(tournament_id=tournament_id, 
                                         t_start_date=start_date,
                                         team_ind=team_ind, 
                                         team_id=team_id, 
                                         players=players, 
                                         mask=mask
                                        )
                                )                    

In [ ]:
del results

In [ ]:
with open("plays_train.pkl", 'wb') as f:
    pickle.dump(plays_train, f)
    
with open("plays_test.pkl", 'wb') as f:
    pickle.dump(plays_test, f)

## Построить baseline-модель

### Выбрать моменты для обучения модели

In [ ]:
%%time
with open("plays_train.pkl", 'rb') as f:
    plays_train = pickle.load(f)
    
with open("plays_test.pkl", 'rb') as f:
    plays_test = pickle.load(f)

In [ ]:
# уникальных игроков в train
len(set([j for i in plays_train for j in i.players]))

In [ ]:
# частота встречаемости игроков в турнирах
frequencies_plays_of_players = Counter([j for i in plays_train for j in i.players]).values()

In [ ]:
# frequencies_plays_of_players := для каждого игрока - сколько раз он играл в 2019
assert len(frequencies_plays_of_players) == len(set([j for i in plays_train for j in i.players]))

In [ ]:
# число игроков с более чем 3мя турнирами
sum(
[num_players for counts, 
     num_players in sorted([(key, value) for key, value in Counter(frequencies_plays_of_players).items()])
if counts >= 3
])

In [ ]:
%%time
train_df = pd.DataFrame(columns=['player', 'tournament_id', 't_start_date', 'team_id', 'mask'])
to_concat = []
for record in tqdm(plays_train):
    for player in record.players:
        to_append = np.array(
            [
            [player, 
             record.tournament_id, 
             record.t_start_date, 
             record.team_id, 
             record.mask]
            ]
        )
        to_concat.append(to_append)
        # train_df = train_df.append(to_append, sort=False, ignore_index=True)

In [ ]:
len(to_concat)

In [ ]:
to_concat = np.vstack(to_concat)

In [ ]:
# %%time
# with open("to_concat.pkl", 'wb') as f:
#     pickle.dump(to_concat, f)

In [ ]:
%%time
with open("to_concat.pkl", 'rb') as f:
    to_concat = pickle.load(f)

In [ ]:
train_df = pd.DataFrame(to_concat, columns=['player', 'tournament_id', 't_start_date', 'team_id', 'mask'])

In [ ]:
train_df = train_df.sort_values(by='player')

In [ ]:
train_df.to_csv("train_df.csv", index=False)

In [ ]:
%%time
test_df = pd.DataFrame(columns=['player', 'tournament_id', 't_start_date', 'team_id', 'mask'])
to_concat = []
for record in tqdm(plays_test):
    for player in record.players:
        to_append = np.array(
            [
            [player, 
             record.tournament_id, 
             record.t_start_date, 
             record.team_id, 
             record.mask]
            ]
        )
        to_concat.append(to_append)
        # test_df = test_df.append(to_append, sort=False, ignore_index=True)

In [ ]:
to_concat = np.vstack(to_concat)

In [ ]:
test_df = pd.DataFrame(to_concat, columns=['player', 'tournament_id', 't_start_date', 'team_id', 'mask'])

In [ ]:
test_df = test_df.sort_values(by='player')

In [ ]:
test_df.to_csv("test_df.csv", index=False)

### Посчитать статистики для обучения на train и обучить модель

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from itertools import product

In [ ]:
%%time
train_df = pd.read_csv("train_df.csv")

In [ ]:
%%time
# проставить target events
train_df = train_df.astype({'player': int,
                            'tournament_id': int,
                            't_start_date': 'datetime64[ns]',
                            'team_id': int,
                            })

train_df.loc[:, 'q_answered'] = train_df['mask'].apply(lambda x: sum([int(i) for i in x if re.match("\d", i)]))
train_df.loc[:, 'q_total'] = train_df['mask'].str.len()

train_df.loc[:, 'target'] = train_df.q_answered / train_df.q_total

print("New train_df.dtypes: \n", "-" * 120, "\n", train_df.dtypes, "\n" ,"-" * 120, sep='')

train_df = train_df.sort_values(by=['player', 't_start_date'])

print("Число уникальных tournament_id", train_df[['tournament_id', 't_start_date']].drop_duplicates().shape)

train_df.loc[:, 'tournament_number'] = train_df.groupby('player').cumcount('tournament_id') + 1

# игроки с более чем 3мя турнирами
players_with_tours = \
train_df.groupby('player').tournament_number.max().loc[train_df.groupby('player').tournament_number.max() > 3].\
index.tolist()

# случайно выбрать моменты для обучения по каждому игроку
moments_to_predict = \
train_df.loc[train_df.player.isin(players_with_tours)].\
groupby('player').tournament_number.max().apply(lambda x: np.random.randint(4, x + 1)).reset_index()

# check uniqness
assert moments_to_predict.shape == moments_to_predict.drop_duplicates().shape

# проставить target_events
moments_to_predict.loc[:, 'target_event'] = 1
moments_to_predict.target_event = moments_to_predict.target_event.fillna(0)

# присоединить метку target_events к основному датасету
train_df = pd.merge(train_df, moments_to_predict, 
         how='left',
         left_on=['player', 'tournament_number'],
         right_on=['player', 'tournament_number']
        )

train_df.loc[:, 'target_event'] = train_df.target_event.fillna(0)

__фичи для обучения__

|Название поля | Способ подсчёта | Описание |
| --- | --- | --- |
|tours_num|tournament_number - 1 | число турниров |
|avg_pct_answered| | средний процент ответов|
|min_pct_answered||| Минимальный процент ответов

In [ ]:
def preprocess_data(df):
    df.loc[:, 'tours_num'] = df.tournament_number - 1

    # среднее
    df.loc[:, "avg_pct_answered"] = \
    (df.groupby("player").q_answered.cumsum() - df.q_answered) / \
    (df.groupby("player").q_total.cumsum() - df.q_total)

    # минимум
    df.loc[:, 'min_pct_answered'] = df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                    min_periods=1).min().values
    df.loc[:, 'min_pct_answered'] = df.groupby("player")['min_pct_answered'].shift(1)


    # максимум
    df.loc[:, 'max_pct_answered'] = df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                    min_periods=1).max().values
    df.loc[:, 'max_pct_answered'] = df.groupby("player")['max_pct_answered'].shift(1)

    # квантили
    df.loc[:, 'pct_answered_q_10'] = df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                    min_periods=1).quantile(0.1).values
    df.loc[:, 'pct_answered_q_10'] = df.groupby("player")['pct_answered_q_10'].shift(1)

    df.loc[:, 'pct_answered_q_90'] = df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                    min_periods=1).quantile(0.9).values
    df.loc[:, 'pct_answered_q_90'] = df.groupby("player")['pct_answered_q_90'].shift(1)

    df.loc[:, 'pct_answered_q_50'] = df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                    min_periods=1).quantile(0.5).values
    df.loc[:, 'pct_answered_q_50'] = df.groupby("player")['pct_answered_q_50'].shift(1)
    return df

In [ ]:
%%time
# посчитать базовые фичи на target events
WINDOW_UPPER_BOUND = 100

train_df = preprocess_data(train_df)

X = \
train_df.loc[train_df.target_event == 1][['tours_num','avg_pct_answered',
         'min_pct_answered',
         'max_pct_answered',
         'pct_answered_q_10',
         'pct_answered_q_50',
         'pct_answered_q_90',
          ]]

y = \
train_df.loc[train_df.target_event == 1].target.values

assert \
train_df.loc[train_df.target_event == 1][['player', 'tournament_id']].shape == \
train_df.loc[train_df.target_event == 1][['player', 'tournament_id']].drop_duplicates().shape

train_df = train_df.set_index(['player', 'tournament_id'])

train_df = train_df.loc[train_df.target_event == 1]

In [ ]:
train_df

In [ ]:
%%time
# посчитать X_train и y_train
X_train = []
y_train = []

for ind, row in tqdm(train_df[['tournament_number',
                         'target_event',
                         'tours_num',
                         'avg_pct_answered',
                         'min_pct_answered',
                         'max_pct_answered',
                         'pct_answered_q_10',
                         'pct_answered_q_90',
                         'pct_answered_q_50',
                         'mask']].iterrows()):
    for x_row, y_row in product([row.values[:-1]], [int(x) for x in row['mask'] if re.match("\d", x)]):
        X_train.append(np.array(x_row))
        y_train = np.append(y_train, y_row)

X_train = np.vstack(X_train)

In [ ]:
# сохранить
# train_X_y = [X_train, y_train]

# with open("train_X_y.pkl", "wb") as f:
#     pickle.dump(train_X_y, f)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

### Аналогично посчитать статистики для теста и проверить модель

In [17]:
from scipy.stats import spearmanr, kendalltau

In [18]:
%%time
test_df = pd.read_csv("test_df.csv")

Wall time: 115 ms


In [19]:
%%time
# проставить target events
test_df = test_df.astype({'player': int,
                            'tournament_id': int,
                            't_start_date': 'datetime64[ns]',
                            'team_id': int,
                            })

test_df.loc[:, 'q_answered'] = test_df['mask'].apply(lambda x: sum([int(i) for i in x if re.match("\d", i)]))
test_df.loc[:, 'q_total'] = test_df['mask'].str.len()

test_df.loc[:, 'target'] = test_df.q_answered / test_df.q_total

print("New test_df.dtypes: \n", "-" * 120, "\n", test_df.dtypes, "\n" ,"-" * 120, sep='')

test_df = test_df.sort_values(by=['player', 't_start_date'])

print("Число уникальных tournament_id", test_df[['tournament_id', 't_start_date']].drop_duplicates().shape)

test_df.loc[:, 'tournament_number'] = test_df.groupby('player').cumcount('tournament_id') + 1

# игроки с более чем 3мя турнирами
players_with_tours = \
test_df.groupby('player').tournament_number.max().loc[test_df.groupby('player').tournament_number.max() > 3].\
index.tolist()

# случайно выбрать моменты для обучения по каждому игроку
moments_to_predict = \
test_df.loc[test_df.player.isin(players_with_tours)].\
groupby('player').tournament_number.max().apply(lambda x: np.random.randint(4, x + 1)).reset_index()

# check uniqness
assert moments_to_predict.shape == moments_to_predict.drop_duplicates().shape

# проставить target_events
moments_to_predict.loc[:, 'target_event'] = 1
moments_to_predict.target_event = moments_to_predict.target_event.fillna(0)

# присоединить метку target_events к основному датасету
test_df = pd.merge(test_df, moments_to_predict, 
         how='left',
         left_on=['player', 'tournament_number'],
         right_on=['player', 'tournament_number']
        )

test_df.loc[:, 'target_event'] = test_df.target_event.fillna(0)

New test_df.dtypes: 
------------------------------------------------------------------------------------------------------------------------
player                    int32
tournament_id             int32
t_start_date     datetime64[ns]
team_id                   int32
mask                     object
q_answered                int64
q_total                   int64
target                  float64
dtype: object
------------------------------------------------------------------------------------------------------------------------
Число уникальных tournament_id (173, 2)
Wall time: 4.37 s


In [20]:
%%time
# посчитать базовые фичи на target events
WINDOW_UPPER_BOUND = 100

test_df.loc[:, 'tours_num'] = test_df.tournament_number - 1

# среднее
test_df.loc[:, "avg_pct_answered"] = \
(test_df.groupby("player").q_answered.cumsum() - test_df.q_answered) / \
(test_df.groupby("player").q_total.cumsum() - test_df.q_total)

# минимум
test_df.loc[:, 'min_pct_answered'] = test_df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                min_periods=1).min().values
test_df.loc[:, 'min_pct_answered'] = test_df.groupby("player")['min_pct_answered'].shift(1)


# максимум
test_df.loc[:, 'max_pct_answered'] = test_df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                min_periods=1).max().values
test_df.loc[:, 'max_pct_answered'] = test_df.groupby("player")['max_pct_answered'].shift(1)

# квантили
test_df.loc[:, 'pct_answered_q_10'] = test_df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                min_periods=1).quantile(0.1).values
test_df.loc[:, 'pct_answered_q_10'] = test_df.groupby("player")['pct_answered_q_10'].shift(1)

test_df.loc[:, 'pct_answered_q_90'] = test_df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                min_periods=1).quantile(0.9).values
test_df.loc[:, 'pct_answered_q_90'] = test_df.groupby("player")['pct_answered_q_90'].shift(1)

test_df.loc[:, 'pct_answered_q_50'] = test_df.groupby('player').target.rolling(WINDOW_UPPER_BOUND, 
                                                                                min_periods=1).quantile(0.5).values
test_df.loc[:, 'pct_answered_q_50'] = test_df.groupby("player")['pct_answered_q_50'].shift(1)

X = \
test_df.loc[test_df.target_event == 1][['tours_num','avg_pct_answered', 
          'min_pct_answered',
         'max_pct_answered',
         'pct_answered_q_10',
         'pct_answered_q_50',
         'pct_answered_q_90',
          ]]

y = \
test_df.loc[test_df.target_event == 1].target.values

assert \
test_df.loc[test_df.target_event == 1][['player', 'tournament_id']].shape == \
test_df.loc[test_df.target_event == 1][['player', 'tournament_id']].drop_duplicates().shape

test_df = test_df.set_index(['player', 'tournament_id'])

test_df = test_df.loc[test_df.target_event == 1]

Wall time: 58 s


In [21]:
# собрать X_test
X_test = test_df[['tournament_number',
                         'target_event',
                         'tours_num',
                         'avg_pct_answered',
                         'min_pct_answered',
                         'max_pct_answered',
                         'pct_answered_q_10',
                         'pct_answered_q_90',
                         'pct_answered_q_50']]

In [ ]:
model.predict_proba(X_test)[:, 1]

In [ ]:
# собрать результаты для сверки
y_test_prob = \
test_df['mask'].apply(lambda x: sum([int(i) for i in x if re.match("\d", i)])) / \
test_df['mask'].apply(lambda x: sum([1 for i in x if re.match("\d", i)]))

In [ ]:
# качество не очень, но это качество между ранжированием игроков, а не ранжированием команд
print("Корреляция Спирмена", spearmanr(model.predict_proba(X_test)[:, 1], y_test_prob.values))
print("Корреляция Кендалла", kendalltau(model.predict_proba(X_test)[:, 1], y_test_prob.values))

- вышеприведённые корреляции применяются для рейтингов игроков, а не для мест команд

### Модель/расчёт для сложности вопросов

In [ ]:
train_df = pd.read_csv("train_df.csv")

In [ ]:
# plt.figure(figsize=(16, 6));
# plt.hist(train_df.groupby("tournament_id").team_id.nunique(), 
#          bins=100, histtype='step', cumulative=True, );
# plt.xticks(np.arange(0, 2000, 50))
# plt.grid(True)
# plt.show()

In [ ]:
# посчитать процент ответов
tournaments_for_train = train_df.groupby("tournament_id").team_id.nunique().\
loc[train_df.groupby("tournament_id").team_id.nunique() > 10].index

tournament_x_question = []
for ind, group in train_df[['tournament_id', 'team_id', "mask"]].drop_duplicates().groupby("tournament_id"):
    # print(group)
    function_to_apply = lambda x: np.array([int(i) for i in x.replace("?", "0") if re.match("\d", i)])
    
    lists = group['mask'].apply(function_to_apply)
    # привести длины в порядок
    lists = list(zip(*lists))
    pct_answered = np.vstack(lists).mean(axis=0)
    
    by_tournament = [(i[0], ind + 1, i[1])  for ind, i in enumerate(product([ind], pct_answered))]
    tournament_x_question.append(by_tournament)


In [ ]:
tournament_x_question

In [ ]:
# посчитаем рейтинг команд
X_train = train_df[['tournament_number',
                         'target_event',
                         'tours_num',
                         'avg_pct_answered',
                         'min_pct_answered',
                         'max_pct_answered',
                         'pct_answered_q_10',
                         'pct_answered_q_90',
                         'pct_answered_q_50']]

predictions = model.predict_proba(X_train)[:, 1]


for_team_rating = train_df[['team_id', 'mask']].reset_index()

for_team_rating.loc[:, 'predicted_rating'] = predictions

team_rating = for_team_rating.groupby(['team_id', 'tournament_id', 'mask']).predicted_rating.mean().reset_index()

In [ ]:
# визуальная проверка на одинаковое число вопросов
# team_rating.assign(mask_length = team_rating['mask'].str.len()).groupby('tournament_id').mask_length.agg(['min','max','mean'])

In [ ]:
# распределение команд по предсказанному рейтингу
plt.figure(figsize=(18, 6))
plt.hist(team_rating.predicted_rating, 
         bins=np.arange(0, 1.1, 0.05), 
         histtype='step',
         cumulative=False,
         density=True,
         linewidth=2
        );
plt.grid(True)
plt.show()

In [ ]:
%%time
q_complexities = []
for tournament_id, group in tqdm(team_rating.groupby('tournament_id')):
    # print(group['mask'].str.len().min())
    for q_number in range(1,  group['mask'].str.len().min() + 1):
        q_complexity = group.loc[group['mask'].apply(lambda x: x[q_number - 1] == '1')].predicted_rating.mean()
        q_complexities.append((tournament_id, q_number, q_complexity))

In [ ]:
q_complexities = pd.DataFrame(q_complexities, columns=['tournament_id', 'q_number', 'q_complexity'])

In [ ]:
q_complexities.to_csv("q_complexities.csv", index=False)

##  Способ ранжирования команд на основе рейтинга игроков

- возьмём числовые характеристики рейтингов игроков в команде и обучим модель на этих рейтингах

__предпосылки:__
- рейтинг игроков якобы не содержит информации о целевой переменной (несмотря на, то, что предпосылка нарушается можно сказать, что рейтинг игроков содержит мало информации о результате конкретной игры и попробовать обучить веса логистической регрессии на характеристиках распределения (min, max, avg, квантили) рейтингов игроков в команде)

### Обучим модель для baseline

In [2]:
train_df = pd.read_csv("train_df.csv")

In [3]:
with open("train_X_y_full.pkl", "rb") as f:
    train_X_y = pickle.load(f)

In [4]:
train_X, train_y = train_X_y

In [5]:
prev_answers_cols = ['tournament_number',
                         'target_event',
                         'tours_num',
                         'avg_pct_answered',
                         'min_pct_answered',
                         'max_pct_answered',
                         'pct_answered_q_10',
                         'pct_answered_q_90',
                         'pct_answered_q_50']
train_X = pd.DataFrame(train_X, 
                       columns= ['player', 'tournament_id', 'team_id'] + prev_answers_cols)

In [6]:
# q_complexities = pd.read_csv("q_complexities.csv")

In [7]:
model1 = LogisticRegression(C=1)
model1.fit(train_X[prev_answers_cols], train_y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
prediction = model1.predict_proba(train_X[prev_answers_cols])[:, 1]

In [9]:
train_X.loc[:, 'player_rating'] = prediction

In [10]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = f"percentile_{n}"
    return percentile_

check = train_X.groupby(["team_id", "tournament_id"]).player_rating.agg(['min', 'max', 'mean', 
                                                                 percentile(10),
                                                                 percentile(90)])
check.columns = ["agg_by_team_" + i for i in check.columns]

- с учётом того, что для 83% команд рейтинг всех игроков равный, можно было бы взять и просто максимальный рейтинг игрока в команде

In [12]:
# с учётом того, что для 83% команд рейтинг всех игроков равный, можно взять и просто максимальный рейтинг игрока в команде
(check['agg_by_team_min'] == check['agg_by_team_max']).value_counts(normalize=True)

True     0.832159
False    0.167841
dtype: float64

In [13]:
tournament_x_team = train_X[['player', 'tournament_id', 'team_id']].copy()

tournament_x_team.loc[:, 'target'] = train_y

tournament_x_team = \
pd.merge(tournament_x_team, check.reset_index(),
         how='inner',
         left_on= ['tournament_id', 'team_id'],
         right_on=['tournament_id', 'team_id'])

X_train = \
tournament_x_team[['agg_by_team_min', 'agg_by_team_max', 'agg_by_team_mean',
       'agg_by_team_percentile_10', 'agg_by_team_percentile_90']]
y_train = tournament_x_team['target']

model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
print("Coefficients:", model2.coef_)
print("Intercept:", model2.intercept_)

Coefficients: [[-1.11345547 -0.39870715  7.40371779 -1.11345547 -0.39870715]]
Intercept: [-2.18913066]


### Протестируем качество модели

In [41]:
# собрать X_test
X_test = test_df.reset_index()[['player', 'tournament_id', 'team_id'] + prev_answers_cols]

In [42]:
test_prediction_model1 = model1.predict_proba(X_test[prev_answers_cols])[:, 1]

In [43]:
X_test.loc[:, 'player_rating'] = test_prediction_model1

In [45]:
agg_test = X_test.groupby(["team_id", "tournament_id"]).player_rating.agg(['min', 'max', 'mean', 
                                                                 percentile(10),
                                                                 percentile(90)])
agg_test.columns = ["agg_by_team_" + i for i in agg_test.columns]

In [47]:
(agg_test['agg_by_team_min'] == agg_test['agg_by_team_max']).value_counts(normalize=True)

True     0.748425
False    0.251575
dtype: float64

In [48]:
tournament_x_team = X_test[['player', 'tournament_id', 'team_id']].copy()

tournament_x_team = \
pd.merge(tournament_x_team, agg_test.reset_index(),
         how='inner',
         left_on= ['tournament_id', 'team_id'],
         right_on=['tournament_id', 'team_id'])

X_test_for_2nd_model = \
tournament_x_team[['agg_by_team_min', 'agg_by_team_max', 'agg_by_team_mean',
       'agg_by_team_percentile_10', 'agg_by_team_percentile_90']]

In [55]:
test_prediction_model2 = model2.predict_proba(X_test_for_2nd_model)[:, 1]

In [65]:
test_prediction_model2.__len__()

8750

In [61]:
# собрать результаты для сверки
y_test_prob = \
test_df['mask'].apply(lambda x: sum([int(i) for i in x if re.match("\d", i)])) / \
test_df['mask'].apply(lambda x: sum([1 for i in x if re.match("\d", i)]))

In [69]:
#проверить, что в рамках одной команды таргеты равны
to_check = test_df.reset_index().groupby(['tournament_id', 'team_id']).target.agg(['min', 'max'])
assert to_check.loc[to_check['min'] != to_check['max']].shape[0] == 0

In [80]:
#проверить, что в рамках одной команды таргеты равны
to_check = test_df.reset_index()[['tournament_id', 'team_id']].copy()

to_check.loc[:, 'prediction'] = test_prediction_model2
to_check = to_check.reset_index().groupby(['tournament_id', 'team_id']).prediction.agg(['min', 'max'])
assert to_check.loc[to_check['min'] != to_check['max']].shape[0] == 0

AssertionError: 

In [63]:
len(y_test_prob)

8750

## Предсказание по EM-алгоритму

## Предсказание сложности вопроса

## Основные выводы